# Playing Different Rhythms (example)

This notebook describes how to create different rhythms, using raw midi-writing capabilities.

## Prerequisites

On mac, the prerequisites are straightforward:
1. `pip install mido`
2. `brew install fluidsynth`
3. `pip install pyfluidsynth`

In [1]:
import numpy as np
import subprocess

# mido imports
import mido
from mido import MidiFile, MidiTrack, Message, MetaMessage

# pyfluidsynth import
import fluidsynth

# IPython imports: for display only, not needed
#from IPython.display import Audio, display



In [2]:
# The standard default tempo (if none is set) is 500,000 µs/qn, which corresponds to 120 BPM.

# You can also set the resolution of the MIDI file (defaults to 480 ticks/beat)

In [3]:


def repeat_note_fixed_span(note=60, velocity=80, total_beats=16, subdivisions=4, program=4, channel=0, duration_list=None,desired_bpm=120):
    """Create a MIDI file with a repeated note pattern over a fixed time span. 
    
    `duration_list` is the coding of the note duration in units of the smallest time unit. That is, if the 'duration' is a sixteenth note, then a 1 entry in duration_list will be a sixteenth note, a 2 will be an eighth note, 4 will be quarter, etc.

    If `sum(duration_list)` exceeds subdivisions*total_beats, the file will just keep going, beyond total_beats.
    """

    total_time = (total_beats // subdivisions) * 480

    if duration_list is None:
        duration_list = [1] * (subdivisions * total_beats)  # Default to a list of ones if not provided

    mid = MidiFile()
    track = MidiTrack()
    mid.tracks.append(track)

    # Set the tempo
    tempo_us_per_qn = 500000  # microseconds per quarter note
    
    bpm = mido.tempo2bpm(tempo_us_per_qn* 120/desired_bpm)
    track.append(MetaMessage('set_tempo', tempo=tempo_us_per_qn, time=0))

    # Set instrument
    track.append(Message('program_change', program=program, channel=channel, time=0))

    # single unit duration
    duration = total_time // subdivisions

    # First note starts at t=0
    track.append(Message('note_on', note=note, velocity=velocity, channel=channel, time=0))
    track.append(Message('note_off', note=note, velocity=64, channel=channel, time=duration*duration_list[0]))

    # Remaining notes
    for i in range(len(duration_list) - 1):
        track.append(Message('note_on', note=note, velocity=velocity, channel=channel, time=0))
        track.append(Message('note_off', note=note, velocity=64, channel=channel, time=duration*duration_list[i+1]))

    return mid

# Example usage:
note_significance = 4  # maximum number of notes to a beat; here we are consider sixteenth notes
total_beats = 8  # total number of beats in the midi


mid = repeat_note_fixed_span(subdivisions=note_significance,total_beats=total_beats,duration_list=[4,1,2,1,1,1,1,1,1,1,1,1,3,1,3,1,2,2,4])
mid.save(f'examplemidis/repeated_7113-6104_fixed.mid')

mid = repeat_note_fixed_span(subdivisions=note_significance,total_beats=total_beats,duration_list=[12,8,12])
mid.save(f'examplemidis/repeated_8444-12703_fixed.mid')

mid = repeat_note_fixed_span(subdivisions=note_significance,total_beats=total_beats)
mid.save(f'examplemidis/repeated_boring_fixed.mid')


In [4]:
# to play live in the notebook, we can use fluidsynth

def midi_to_audio_data(midi_path, sf2_path, samplerate=44100):
    fs = fluidsynth.Synth(samplerate=samplerate)
    fs.start(driver='coreaudio')  # for systems without 'file' driver
    sfid = fs.sfload(sf2_path)
    fs.program_select(0, sfid, 0, 0)

    mid = MidiFile(midi_path)
    
    # Allocate a large buffer for audio
    audio_buffer = []

    for msg in mid.play():
        if msg.is_meta:
            continue
        elif msg.type == 'note_on':
            fs.noteon(msg.channel, msg.note, msg.velocity)
        elif msg.type == 'note_off':
            fs.noteoff(msg.channel, msg.note)
        elif msg.type == 'program_change':
            fs.program_change(msg.channel, msg.program)

        # Each MIDI message advances time, so grab audio output chunk
        audio_chunk = fs.get_samples(512)
        audio_buffer.extend(audio_chunk)

    fs.delete()

    # Convert to numpy array and normalise to [-1, 1]
    audio = np.array(audio_buffer, dtype=np.float32)
    audio = audio.reshape(-1, 2)  # stereo
    audio = np.clip(audio, -32768, 32767) / 32768.0

    return audio, samplerate

# if wanting to make a widget to play the MIDI file:
#def play_midi(midi_path, sf2_path):
#    audio_data, sr = midi_to_audio_data(midi_path, sf2_path)
#    display(Audio(audio_data.T, rate=sr))  # Transpose for (channels, samples)

# or if we just want to save, we can use this!
def midi_to_wav(midi_file, wav_file, sf2_file, sample_rate=44100):

    cmd = [
        "fluidsynth",
        "-ni",                 # no shell, play immediately
        sf2_file,
        midi_file,
        "-F", wav_file,      # write to WAV file
        "-r", "44100"          # sample rate
    ]

    subprocess.run(cmd)

In [6]:
# Usage
midi_path = "examplemidis/repeated_8444-12703_fixed.mid"
#midi_path = "examplemidis/repeated_7113-6104_fixed.mid"
midi_path = "examplemidis/repeated_boring_fixed.mid"

soundfont_path = "soundfonts/FluidR3_GM.sf2"

# this plays the midi file in the notebook
audio_data, sr = midi_to_audio_data(midi_path, soundfont_path)

# this saves the midi file as a wav file
wav_file = 'examplemidis/repeated_8444-12703_fixed.wav'
midi_to_wav(midi_path, wav_file, soundfont_path, sample_rate=44100)


FluidSynth runtime version 2.4.4
Copyright (C) 2000-2025 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file 'examplemidis/repeated_8444-12703_fixed.wav'..
